# Import and Initialization

In [53]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf

# standard library imports
from __future__ import absolute_import, division, print_function

# standard numerical library imports
import numpy as np

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, remap_pids, to_categorical

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


import pandas
import h5py
import pickle

import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# configs
train, val, test = 0.7, 0.2, 0.1
Phi_sizes, F_sizes = (256, 256, 256), (256, 256, 256)
num_epoch = 500
batch_size = 2048

# Load and Process Data

In [3]:
b_signal_file = "/global/home/users/yifengh3/VAE/data/B_signal.h5"
background_file = "/global/home/users/yifengh3/VAE/data/B_background.h5"
b_signal_data = pandas.read_hdf(b_signal_file)
background_data = pandas.read_hdf(background_file)

In [4]:
print("backgroubackground_datand data shape: {}".format(background_data.shape))
print("B signal data shape: {}".format(b_signal_data.shape))

backgroubackground_datand data shape: (2000000, 150)
B signal data shape: (50000, 150)


In [7]:
# assign labels to signal and background data, 0 for background,  1 for b-signal
# (updated since we might get multiple signals) 
labeled_background_data = np.append(background_data,np.zeros((background_data.shape[0],1)),axis=1)
labeled_b_signal_data = np.append(b_signal_data,np.ones((b_signal_data.shape[0],1)),axis=1)

In [8]:
# mix two data array into one signal array
data = np.concatenate((labeled_b_signal_data,labeled_background_data))

#and shuffle the data
np.random.shuffle(data)

In [9]:
X = data[:,:-1]
y = data[:,-1]

In [10]:
print("shape of X: {}".format(X.shape))
print("shape of Y: {}".format(y.shape))

shape of X: (2050000, 150)
shape of Y: (2050000,)


In [11]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
total = b_signal_data.shape[0] + background_data.shape[0]
weight_for_0 = (1 / background_data.shape[0]) * (total / 2.0)
weight_for_1 = (1 / b_signal_data.shape[0]) * (total / 2.0)


class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for background: {:.2f}'.format(weight_for_0))
print('Weight for signal: {:.2f}'.format(weight_for_1))

Weight for background: 0.51
Weight for signal: 20.50


In [12]:
# To categorical as stipulated in example
Y = to_categorical(y, num_classes=2)

# Reshape X to shape (number of jets, 50, 3)
X = X.reshape(-1,50,3)

In [13]:
# normalizing jets
# copied from example
import tqdm
for x in tqdm.tqdm(X):
    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

100%|██████████| 2050000/2050000 [01:58<00:00, 17335.89it/s]


In [14]:
print('Finished preprocessing')
print("shape of X: {}".format(X.shape))
print("shape of Y: {}".format(y.shape))

Finished preprocessing
shape of X: (2050000, 50, 3)
shape of Y: (2050000,)


In [15]:
# do train/val/test split 
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=val, test=test)

# Build and Train the Model

In [90]:
print('Model summary:')

# build architecture
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes)

Model summary:
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 256)    1024        input[0][0]                      
__________________________________________________________________________________________________
activation_63 (Activation)      (None, None, 256)    0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 256)    65792       activation_63[0][0]              
_____________________________________________________________________________

In [ ]:
# now train the model

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1**(1/5), patience=5, min_lr=1e-4,
                                                verbose=1)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, 
                                              verbose=1)

callbacks = [reduce_lr,early_stop]

hist = pfn.fit(X_train, Y_train,
        epochs=num_epoch,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        class_weight=class_weight,
        callbacks=callbacks,
        verbose=1)

Epoch 1/500
701/701 [==============================] - 12s 16ms/step - loss: 8.4193 - acc: 0.5062 - val_loss: 0.4630 - val_acc: 0.9512
Epoch 2/500
701/701 [==============================] - 11s 16ms/step - loss: 0.7060 - acc: 0.5300 - val_loss: 1.0496 - val_acc: 0.0427
Epoch 3/500
701/701 [==============================] - 11s 16ms/step - loss: 0.7100 - acc: 0.5132 - val_loss: 0.6790 - val_acc: 0.5931
Epoch 4/500
701/701 [==============================] - 11s 16ms/step - loss: 0.6954 - acc: 0.5475 - val_loss: 0.4326 - val_acc: 0.9639
Epoch 5/500
701/701 [==============================] - 11s 16ms/step - loss: 0.6970 - acc: 0.5300 - val_loss: 0.8528 - val_acc: 0.0304
Epoch 6/500
701/701 [==============================] - 11s 16ms/step - loss: 0.6951 - acc: 0.5330 - val_loss: 0.6051 - val_acc: 0.8895
Epoch 7/500
701/701 [==============================] - 11s 16ms/step - loss: 0.6958 - acc: 0.5561 - val_loss: 0.6845 - val_acc: 0.6458
Epoch 8/500
701/701 [==============================] - 

# Analyze the Model

In [87]:
# get predictions on test data
preds = pfn.predict(X_test, batch_size=10000)

# get ROC curve
pfn_fp, pfn_tp, threshs = roc_curve(Y_test[:,1], preds[:,1])

# get area under the ROC curve
auc = roc_auc_score(Y_test[:,1], preds[:,1])
print()
print('PFN AUC:', auc)
print()


PFN AUC: 0.4999004380724811



In [ ]:
# get multiplicity and mass for comparison
masses = np.asarray([ef.ms_from_p4s(ef.p4s_from_ptyphims(x).sum(axis=0)) for x in X])
mults = np.asarray([np.count_nonzero(x[:,0]) for x in X])
mass_fp, mass_tp, threshs = roc_curve(Y[:,1], -masses)
mult_fp, mult_tp, threshs = roc_curve(Y[:,1], -mults)

# some nicer plot settings 
plt.rcParams['figure.figsize'] = (4,4)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.autolayout'] = True

# plot the ROC curves
plt.plot(pfn_tp, 1-pfn_fp, '-', color='black', label='PFN')
plt.plot(mass_tp, 1-mass_fp, '-', color='blue', label='Jet Mass')
plt.plot(mult_tp, 1-mult_fp, '-', color='red', label='Multiplicity')

# axes labels
plt.xlabel('Quark Jet Efficiency')
plt.ylabel('Gluon Jet Rejection')

# axes limits
plt.xlim(0, 1)
plt.ylim(0, 1)

# make legend and show plot
plt.legend(loc='lower left', frameon=False)
plt.show()
